In [ ]:
import re
from itertools import product 
import string
import random
import numpy as np
from os import listdir
from os.path import isfile, join
from unidecode import unidecode
from tqdm import notebook as nb
from numpy import dot
from numpy.linalg import norm
from scipy import spatial
import sys
from tqdm.notebook import  tqdm
from numpy import dot
import random
import math 
import copy
from numpy.linalg import norm

NumOfClass = 16;
ClassLabel= ["acq_1","acq_2","crude1","crude2","earn1","earn2","grain1","grain2","money-fx1","money-fx2","interest1","interest2","ship1","ship2","trade1","trade2"]
ClassMap = ["ac","ac","cr","cr","ea","ea","gr","gr","mo","mo","in","in","sh","sh","tr","tr"]

# NumOfClass = 4
# ClassLabel= ["grain1","grain2","ship1","ship2"]
# ClassMap = ["gr","gr","sh","sh"]

# NumOfClass = 1
# ClassLabel= ["acq_1"]
# ClassMap = ["ac"]

DIMENSION = 10000
CHARACTERS = 256

def norm_values(vec, n):
    result = norm(vec)
    return result


def cosAngle_val(u, v, n):
    dot_prod = dot(u, v)    
    temp = (norm_values(u,n)*norm_values(v,n))
    result = dot_prod/temp
    return result


def randPerm(SZ):
    perm = [0] * SZ

    for i in range(SZ):
        perm[i] = i
    
    for i in range(SZ):
        j=0
        t=0
        j = random.randint(0, 32767) % (SZ-i) + i
        t = perm[j]
        perm[j] = perm[i]
        perm[i] = t
    return perm


def RandomHV(randomHV):
    i=0
    perm = randPerm(DIMENSION)
    for i in range(int(DIMENSION/2)):
        randomHV[perm[i]] = 1
    
    for i in range(int(DIMENSION/2), DIMENSION):
        randomHV[perm[i]] = -1
    return randomHV

def check_ItemMemory(itemMemory, key):
    if not key in itemMemory:
        itemMemory[key]  = [0] * DIMENSION
        itemMemory[key] = RandomHV(itemMemory[key])
    return itemMemory[key]


def binary_HV(v, size):
    i = 0
    threshold = 0
    for i in range(size):
        if v[i] > threshold:
            v[i] = 1
        else:
            v[i] = -1


def permute_val_back(ind, xdim, ydim):
    i = 0
    j = 0
    temp = ind[xdim-1].copy()
    print(temp)
    for i in reversed(range(1, xdim)):
        ind[i] = ind[i-1]
    
    print(temp)
    ind[0] = temp
    for i in range(xdim):
        tempj = ind[i][ydim-1].copy()
    
        for j in reversed(range(1, ydim)):
            ind[i][j] = ind[i][j-1]
        ind[i][0] = tempj

def permute_val(ind, xdim, ydim):
    np.roll(ind, [1, 1], axis=(0, 1))

def computeHvForWord(itemMemory, tempWord, wordLength, sumHV):
    isIMChanged = 0
    nGrams = None
    tempBlock = None
    tempBlock = [0] * wordLength
    
    for i in range(wordLength):
        tempBlock[i] = [0] * DIMENSION
    

    character = ''
    nGrams =  [0] * DIMENSION

    for wordHead in range(wordLength):
        character = tempWord[wordHead]
        permute_val(tempBlock, wordLength, DIMENSION)
        if not character in itemMemory:
            isIMChanged = 1

        tempHv = check_ItemMemory(itemMemory, character)
        tempBlock[0] = copy.deepcopy(tempHv)
        nGrams = copy.deepcopy(tempBlock[0])
        
        for i in range(1, wordLength):
            nGrams = np.multiply(nGrams, tempBlock[i])
            sumHV = np.add(sumHV, nGrams)
#             print("+++++++++++++++++++")
#             print(nGrams_x)
#             print(sumHV_y)
#             for j in range(DIMENSION):
#                 nGrams[j] *= tempBlock[i][j]
#                 sumHV[j] += nGrams[j]
#             print("=======================")
#             print(nGrams)
#             print(sumHV)
    return itemMemory

def compute_topicHV(text, itemMemory, associativeMemory, sumHV):
    isIMChanged = 0
    wordLength = 0
    tempWord = [None] * 15
    toolbar_width = len(text)
    with tqdm(total=toolbar_width) as pbar:
        for head in range(len(text)):
            if text[head] != ' ' and wordLength < 14:
                tempWord[wordLength] = text[head]
                wordLength = wordLength + 1
                continue
            elif wordLength > 1 and wordLength < 14:
                itemMemory = computeHvForWord(itemMemory, tempWord, wordLength, sumHV)
                tempWord = [None] * 15

            for i in range(15):
                tempWord[i] = None
            wordLength = 0
            pbar.update(head - pbar.n)

    return isIMChanged


def buildtext(itemMemory, associativeMemory):
    i = 0
    length = 0
    for i in range(NumOfClass):
        path = 'train_double'
        path = path + "/" + ClassLabel[i] + ".txt"
        with open(path, encoding='utf-8') as f:
            content = f.read()
            print("Training file %s \n", path)
            associativeMemory[i] = [0] * DIMENSION
            compute_topicHV(content, itemMemory, associativeMemory, associativeMemory[i])

            
def test_files(langAM, itemMemory):
    total = 0
    correct = 0
    accuracy = 0

    actLabel = ''
    preLang = ''
    i = 0
        
    textHV = [0] * DIMENSION

    angle = 0
    maxAngle = 0
    k = 0
    actLabel = ''
    preLang = ''
    
    test_path="test"   
    onlyfiles = [f for f in listdir(test_path) if isfile(join(test_path, f))]
        
    for file in onlyfiles:
        with open(test_path+"/"+file, encoding='utf-8') as f:
            print(file)
            actLabel = file
            content = f.read()
            isChanged = compute_topicHV(content, itemMemory, associativeMemory, textHV)
            binary_HV(textHV, DIMENSION)
            if isChanged:
                print("Error in File \n")
                return

            maxAngle = 0
            for i in range(NumOfClass):
                angle = abs(cosAngle_val(langAM[i], textHV, DIMENSION))
                if angle > maxAngle:
                    maxAngle = angle
                    preLang = ClassMap[i]
            print(preLang)
            print(actLabel[:2])
            if preLang == actLabel[:2]:
                correct += 1
            else:
                print("No Match: %s --> %s \n", actLabel, preLang)
            total = total + 1
        
    accuracy = correct/total * 100
    print("Accuracy = %f percent\n", accuracy)


# if __name__ == '__main__':
    
#     associativeMemory = [0] * NumOfClass
#     itemMemory = {}
    
#     buildtext(itemMemory, associativeMemory)
    
#     print(associativeMemory)
#     print(itemMemory)
#     for x in range(NumOfClass):
#         for y in range(DIMENSION):
#             print(associativeMemory[x][y])

#     test_files(associativeMemory, itemMemory)

In [40]:
def test_files(langAM, itemMemory):
    total = 0
    correct = 0
    accuracy = 0

    actLabel = ''
    preLang = ''
    i = 0
        
    textHV = [0] * DIMENSION

    angle = 0
    maxAngle = 0
    k = 0
    actLabel = ''
    preLang = ''
    
    test_path="test"   
    onlyfiles = [f for f in listdir(test_path) if isfile(join(test_path, f))]
        
    for file in onlyfiles:
        with open(test_path+"/"+file, encoding='utf-8') as f:
            print(file)
            actLabel = file
            content = f.read()
            isChanged = compute_topicHV(content, itemMemory, associativeMemory, textHV)
            binary_HV(textHV, DIMENSION)
            if isChanged:
                print("Error in File \n")
                return

            maxAngle = 0
            for i in range(NumOfClass):
                angle = abs(cosAngle_val(langAM[i], textHV, DIMENSION))
                if angle > maxAngle:
                    maxAngle = angle
                    preLang = ClassMap[i]
            print(preLang)
            print(actLabel[:2])
            if preLang == actLabel[:2]:
                correct += 1
            else:
                print("No Match: %s --> %s \n", actLabel, preLang)
            total = total + 1
        
    accuracy = correct/total * 100
    print("Accuracy = %f percent\n", accuracy)


In [6]:
### for saving the trained language vector to a file
import pickle5
def save_obj(obj, name ):
    with open( 'obj/'+ name + '.pkl', 'wb') as f:
        pickle5.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle5.load(f)  

In [7]:
itemMemory = load_obj("news_8_languages_item_memory")
associativeMemory = load_obj("news_8_languages_associative_memory")

In [76]:
x = np.arange(49)
x2 = np.reshape(x, (7,7))
permute_val(x2, 7,7)
x2

[42 43 44 45 46 47 48]
[42 43 44 45 46 47 48]


array([[48, 42, 43, 44, 45, 46, 47],
       [ 6,  0,  1,  2,  3,  4,  5],
       [13,  7,  8,  9, 10, 11, 12],
       [20, 14, 15, 16, 17, 18, 19],
       [27, 21, 22, 23, 24, 25, 26],
       [34, 28, 29, 30, 31, 32, 33],
       [41, 35, 36, 37, 38, 39, 40]])

In [79]:
x3 = np.arange(7)
x3

array([0, 1, 2, 3, 4, 5, 6])

In [80]:
print(x3 * x2[0])

[  0  42  86 132 180 230 282]


In [82]:
np.multiply(x3, x2[0])

array([  0,  42,  86, 132, 180, 230, 282])

In [83]:
np.add(x3, x2[0])

array([48, 43, 45, 47, 49, 51, 53])

In [72]:
x = np.arange(49)
x2 = np.reshape(x, (7,7))
print(x2)
np.roll(x2, [1, 1], axis=(0, 1))

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]
 [35 36 37 38 39 40 41]
 [42 43 44 45 46 47 48]]


array([[48, 42, 43, 44, 45, 46, 47],
       [ 6,  0,  1,  2,  3,  4,  5],
       [13,  7,  8,  9, 10, 11, 12],
       [20, 14, 15, 16, 17, 18, 19],
       [27, 21, 22, 23, 24, 25, 26],
       [34, 28, 29, 30, 31, 32, 33],
       [41, 35, 36, 37, 38, 39, 40]])

In [93]:
test_files(associativeMemory, itemMemory)

acq0.txt



ac
ac
acq1.txt



ac
ac
acq10.txt


KeyboardInterrupt: 

In [94]:
test_files(associativeMemory, itemMemory)

acq0.txt



ac
ac
acq1.txt



ac
ac
acq10.txt



ac
ac
acq100.txt



ac
ac
acq101.txt


...


ac
ac
acq104.txt



ac
ac
acq105.txt



ac
ac
acq106.txt



ac
ac
acq107.txt


KeyboardInterrupt: 

In [95]:
class HDModel():
    def __init__(self, item_memory, associate_memory):
        self.item_memory = item_memory
        self.associate_memory = associate_memory
        
    def genTextVector(self, text):
        textHV = [0] * DIMENSION
        isChanged = compute_topicHV(text, itemMemory, associativeMemory, textHV)
        return textHV
        
    def load_obj(self, name):
        with open('obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)

In [100]:
import pandas as pd
import pickle
import random

from textattack.models.wrappers import ModelWrapper
from joblib import Parallel, delayed


class HDModelWrapper(ModelWrapper):
    """Loads a scikit-learn model and tokenizer (tokenizer implements
    `transform` and model implements `predict_proba`).
    May need to be extended and modified for different types of
    tokenizers.
    """

    def __init__(self, model):
        self.model = model
        
    def __call__(self, text_input_list, batch_size=None):
        results_out = []
        for text in text_input_list:
            textHV = self.model.genTextVector(text)
            binary_HV(textHV, DIMENSION)
            results_temp = []
            for i in range(0, NumOfClass, 2):
                angle_1 = abs(cosAngle_val(self.model.associate_memory[i], textHV, DIMENSION))
                angle_2 = abs(cosAngle_val(self.model.associate_memory[i+1], textHV, DIMENSION))
                if angle_1 > angle_2:
                    results_temp.append(angle_1)
                else:
                    results_temp.append(angle_2)
                
            results_out.append(results_temp)
            print(results_temp)
#         results_out = np.array(results_out)
        return results_out
        
    def get_grad(self, text_input):
        raise NotImplementedError()

In [101]:
model = HDModel(itemMemory, associativeMemory)
model_wrapper = HDModelWrapper(model)

In [98]:
import textattack

text = "anheuser busch joins bid for san miguel anheuser busch companies inc bud n has joined several other foreign bidders for sequestered shares of the philippines largest food and beverage maker san miguel corp sanm mn the head of a government panel which controls the shares told reuters ramon diaz secretary of the presidential commission on good government pcgg said anheuser busch had told the government it was interested in buying mln b shares of san miguel he did not disclose the offered price diaz said australian brewer alan bond s bond corp holdings ltd had offered pesos per share for the b shares diaz said new york investment bank allen and co inc had earlier said it was interested in buying all mln sequestered shares he told reuters last month elders ixl ltd elxa s the melbourne based brewing company had also bid for the b shares the hong kong economic journal last month quoted a spokesman of australian stock broker jacksons ltd as saying that barwon farmlands ltd an australian firm owned pct by ariadne australia ltd was planning a filipino branch in order to buy the entire block of mln shares anheuser busch last year made a mln dlr bid to buy san miguel brewery ltd a hong kong listed company which is pct owned by neptunia corp ltd a san miguel corp subsidiary the talks broke down last june after the two sides said they could not agree on the terms of the sale reuter"
print(len(text.split()))
data = [(text, 0)]

dataset = textattack.datasets.Dataset(data)

248


In [102]:
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019, DeepWordBugGao2018, TextBuggerLi2018, GeneticAlgorithmAlzantot2018, PWWSRen2019
from textattack import Attacker
from textattack import AttackArgs

# dataset = custom_dataset
attack = DeepWordBugGao2018.build(model_wrapper)

# dataset = HuggingFaceDataset("newsgroup", None, "test")

attack_args = AttackArgs(num_examples=1, log_to_csv="log.csv")
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()



textattack: Unknown if model of class <class '__main__.HDModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log.csv

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 




[0.007910862400633715, 0.00448796995858098, 0.006772553451412562, 0.0023690622971027156, 0.0009352933180152169, 0.0014853041766527504, 0.0020745599842239898, 0.007099364417869995]



[0.007910862400633715, 0.00448796995858098, 0.006772553451412562, 0.0023690622971027156, 0.0009352933180152169, 0.0014853041766527504, 0.0020745599842239898, 0.007099364417869995]



[0.007910862400633715, 0.00448796995858098, 0.006772553451412562, 0.0023690622971027156, 0.0009352933180152169, 0.0014853041766527504, 0.0020745599842239898, 0.007099364417869995]


...


[0.007910862400633715, 0.00448796995858098, 0.006772553451412562, 0.0023690622971027156, 0.0009352933180152169, 0.0014853041766527504, 0.0020745599842239898, 0.007099364417869995]



[0.007910862400633715, 0.00448796995858098, 0.006772553451412562, 0.0023690622971027156, 0.0009352933180152169, 0.0014853041766527504, 0.0020745599842239898, 0.007099364417869995]



100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [15:19:21<00:00, 55161.62s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1: 100%|████████████████████████| 1/1 [15:19:22<00:00, 55162.07s/it]

--------------------------------------------- Result 1 ---------------------------------------------
0 (13%) --> [FAILED]

anheuser busch joins bid for san miguel anheuser busch companies inc bud n has joined several other foreign bidders for sequestered shares of the philippines largest food and beverage maker san miguel corp sanm mn the head of a government panel which controls the shares told reuters ramon diaz secretary of the presidential commission on good government pcgg said anheuser busch had told the government it was interested in buying mln b shares of san miguel he did not disclose the offered price diaz said australian brewer alan bond s bond corp holdings ltd had offered pesos per share for the b shares diaz said new york investment bank allen and co inc had earlier said it was interested in buying all mln sequestered shares he told reuters last month elders ixl ltd elxa s the melbourne based brewing company had also bid for the b shares the hong kong economic journal la


c:\users\20522516\appdata\local\programs\python\python37\lib\site-packages\textattack\loggers\attack_log_manager.py:135: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = perturbed_word_percentages.mean()
c:\users\20522516\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
